simple form processing
=========
简单的表单处理

# 写一个简单的表单

更新前面的polls/detail模板（“polls / detail.html”），以便模板包含一个HTML <form>元素：

**polls/templates/polls/detail.html**
```
<h1>{{ question.question_text }}</h1>

{% if error_message %}<p><strong>{{ error_message }}</strong></p>{% endif %}

<form action="{% url 'polls:vote' question.id %}" method="post">
{% csrf_token %}
{% for choice in question.choice_set.all %}
    <input type="radio" name="choice" id="choice{{ forloop.counter }}" value="{{ choice.id }}" />
    <label for="choice{{ forloop.counter }}">{{ choice.choice_text }}</label><br />
{% endfor %}
<input type="submit" value="Vote" />
</form>
```

**简要说明：**

> 上面的模板为每个问题选项显示一个单选按钮。每个单选按钮的值是关联的问题选项的ID。每个单选按钮的名称是“选择”。这意味着，当有人选择其中一个单选按钮并提交表单时，它会发送POST数据选择=＃，其中＃是所选选项的ID。这是HTML表单的基本概念。

> 我们将表单的行为设置为{％url'polls：vote'question.id％}，并设置method =“post”。使用method =“post”（与method =“get”相对）非常重要，因为提交此表单的行为将会改变数据服务器端的行为。每当你创建一个改变数据服务器端的表单时，使用method =“post”。这个提示并不是针对Django的;这只是一个很好的Web开发实践。

> forloop.counter表示for标签通过循环的次数

> 由于我们正在创建POST表单（可能会影响修改数据），因此我们需要担心跨站点请求伪造。值得庆幸的是，您不必太担心，因为Django带有一个非常易用的系统来保护它。简而言之，所有面向内部URL的POST表单都应使用{％csrf_token％}模板标记。

现在，让我们创建一个处理提交数据的Django视图，并对其进行处理。 请记住，在教程3中，我们为包含以下行的`polls`应用程序创建了一个URLconf：

**polls/urls.py**

`path('<int:question_id>/vote/', views.vote, name='vote'),`

我们还创建了vote（）函数的虚拟实现。 我们来创建一个真正的版本。 将以下内容添加到polls / views.py：

**polls/views.py**
```
from django.shortcuts import get_object_or_404, render
from django.http import HttpResponseRedirect, HttpResponse
from django.urls import reverse

from .models import Choice, Question
# ...
def vote(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    try:
        selected_choice = question.choice_set.get(pk=request.POST['choice'])
    except (KeyError, Choice.DoesNotExist):
        # Redisplay the question voting form.
        return render(request, 'polls/detail.html', {
            'question': question,
            'error_message': "You didn't select a choice.",
        })
    else:
        selected_choice.votes += 1
        selected_choice.save()
        # Always return an HttpResponseRedirect after successfully dealing
        # with POST data. This prevents data from being posted twice if a
        # user hits the Back button.
        return HttpResponseRedirect(reverse('polls:results', args=(question.id,)))
 ```

这段代码包含了本教程中尚未涉及的一些内容：

>request.POST是一个类似字典的对象，允许您通过键名访问提交的数据。在这种情况下，request.POST ['choice']以字符串形式返回所选选项的ID。 request.POST值始终是字符串。

>请注意，Django还提供request.GET以相同的方式访问GET数据 - 但我们明确在代码中使用了request.POST，以确保数据仅通过POST调用进行更改。

>如果在POST数据中未提供选项，request.POST ['choice']将引发KeyError。上面的代码检查KeyError并重新显示带有错误消息的问题表单，如果没有给出选择。

>增加选择计数后，代码返回HttpResponseRedirect而不是正常的HttpResponse。 HttpResponseRedirect接受一个参数：用户将被重定向到的URL（在这种情况下，请参阅以下关于如何构造URL的内容）。

>正如上面的Python注释所指出的那样，您应该总是在成功处理POST数据后返回HttpResponseRedirect。这个提示并不是针对Django的;这只是一个很好的Web开发实践。

>在本例中，我们在HttpResponseRedirect构造函数中使用reverse（）函数。此功能有助于避免在视图功能中硬编码URL。它给出了我们想要传递控制权的视图的名称以及指向该视图的URL模式的可变部分。在这种情况下，使用我们在Tutorial 3中设置的URLconf，这个reverse（）调用将返回一个类似的字符串
>>`'/polls/3/results/'`

>其中3是question.id的值。 然后，这个重定向的网址将调用“结果”视图来显示最终页面。

正如前面的教程所述，request是一个HttpRequest对象。 有关HttpRequest对象的更多信息，请参阅[request and response documentation.](https://docs.djangoproject.com/en/2.0/ref/request-response/)。

有人在question中投票后，vote（）视图将重定向到问题的结果页面。 我们来写下这个视图：

**polls/views.py**
```
from django.shortcuts import get_object_or_404, render


def results(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    return render(request, 'polls/results.html', {'question': question})
```

这与前面的detail（）视图几乎完全一样。唯一的区别是模板名称。我们稍后将修复此冗余。

现在，创建一个polls / results.html模板：

**polls/templates/polls/results.html**
```
<h1>{{ question.question_text }}</h1>

<ul>
{% for choice in question.choice_set.all %}
    <li>{{ choice.choice_text }} -- {{ choice.votes }} vote{{ choice.votes|pluralize }}</li>
{% endfor %}
</ul>

<a href="{% url 'polls:detail' question.id %}">Vote again?</a>
```

现在，在您的浏览器中转到/ polls / 1 /并对问题进行投票。 您应该会看到每次投票时都会更新的结果页面。 如果您在未选择选项的情况下提交表单，则应该看到错误消息。

**注意**

我们的vote（）视图的代码确实有一个小问题。 它首先从数据库中获取selected_choice对象，然后计算投票的新值，然后将其保存回数据库。如果您的网站的两个用户试图在同一时间投票，这可能会出错：相同的值，比方说42，将被检索以获得投票。

然后，为两个用户计算并保存43的新值，但44将是预期值。这被称为紊乱情况。 如果你有兴趣，你可以阅读使用F（）避免紊乱情况，以了解如何解决这个问题。

# 使用类视图减少重复的代码

`detail（）`和results（）视图非常简单 - 如上所述，冗余。 显示polls列表的index（）视图与此类似。

这些视图代表了基本Web开发的常见情况：根据URL中传递的参数从数据库获取数据，加载模板并返回呈现的模板。 由于这很常见，Django提供了一个称为“通用视图”系统的快捷方式。

类视图将常见模式抽象到您甚至不需要编写Python代码来编写应用程序的地步。

我们将我们的投票应用程序转换为使用通用视图系统，以便我们可以删除一大堆我们自己的代码。 我们只需采取几个步骤即可完成转换。 我们会：

1. 转换URLconf。
2. 删除一些旧的不需要的视图。
3. 基于Django的通用视图引入新的视图。

详情请阅读。

**为什么来回改动代码？**

通常在写一个Django的app时，我们一开始就要决定使用类视图还是不用，而不是等到代码写到一半了才重构你的代码成类视图。但是本教程为了让你清晰的理解视图的内涵，“故意”走了一条比较曲折的路，因为我们的哲学是在你使用计算器之前你得先知道基本的数学公式。

# 修改URLconf

首先，打开polls / urls.py URLconf并像下面这样改变它：

**polls/urls.py**
```
from django.urls import path

from . import views

app_name = 'polls'
urlpatterns = [
    path('', views.IndexView.as_view(), name='index'),
    path('<int:pk>/', views.DetailView.as_view(), name='detail'),
    path('<int:pk>/results/', views.ResultsView.as_view(), name='results'),
    path('<int:question_id>/vote/', views.vote, name='vote'),
]
```

请注意，第二个和第三个模式的路径字符串中匹配模式的名称已从<question_id>更改为<pk>。

# 修改view

接下来，我们将删除旧的索引，详细信息和结果视图，并使用Django的通用视图。 为此，请打开polls / views.py文件并像下面这样更改它：

**polls/views.py**
```
from django.shortcuts import get_object_or_404, render
from django.http import HttpResponseRedirect
from django.urls import reverse
from django.views import generic

from .models import Choice, Question


class IndexView(generic.ListView):
    template_name = 'polls/index.html'
    context_object_name = 'latest_question_list'

    def get_queryset(self):
        """Return the last five published questions."""
        return Question.objects.order_by('-pub_date')[:5]


class DetailView(generic.DetailView):
    model = Question
    template_name = 'polls/detail.html'


class ResultsView(generic.DetailView):
    model = Question
    template_name = 'polls/results.html'


def vote(request, question_id):
    ... # same as above, no changes needed.

```

我们在这里使用两个类视图：ListView和DetailView。 这两个视图分别抽象出“显示对象列表”和“显示特定类型对象的详细页面”的概念。

>每个类视图都需要知道它将采取何种模式。 这是使用模型属性提供的。
>DetailView 类视图期望从URL中捕获的主键值被称为“pk”，所以我们已将question_id更改为类视图的pk。

默认情况下，DetailView通用视图使用名为<app name> / <model name> _detail.html的模板。在我们的例子中，它会使用模板“polls / question_detail.html”。 template_name属性用于告诉Django使用特定的模板名称而不是自动生成的默认模板名称。我们还为结果列表视图指定了template_name - 这确保了结果视图和细节视图在呈现时具有不同的外观，即使它们在幕后都是DetailView。

同样，ListView通用视图使用名为<应用程序名称> / <模型名称> _list.html的默认模板;我们使用template_name来告诉ListView使用我们现有的“polls / index.html”模板。

在本教程的前几部分中，模板已经提供了一个包含question和latest_question_list上下文变量的上下文。对于DetailView，问题变量是自动提供的 - 由于我们使用Django模型（问题），Django能够为上下文变量确定合适的名称。但是，对于ListView，自动生成的上下文变量是question_list。为了覆盖这个，我们提供context_object_name属性，指定我们想用latest_question_list来代替。作为一种替代方法，您可以更改模板以匹配新的默认上下文变量 - 但只要告诉Django使用您想要的变量就容易多了。

运行服务器，并根据通用视图使用新的轮询应用程序。

有关通用视图的完整详细信息，请参阅 [generic views documentation.](https://docs.djangoproject.com/en/2.0/topics/class-based-views/)。

熟悉了表单和通用视图，下一部分将了解如何测试 `polls` 应用程序。